In [3]:
import numpy as np
import numbers
from scipy.misc import comb
import ink

def _is_integer(val):
    return isinstance(val, (int, np.int32, np.int64))

# Python 3.5

[Used paper](http://sci-hub.io/10.1109/IJCNN.2013.6706860)

$$
K(x, y) = \sum_{r=0}^{d} (x-a)^r(y-a)^r + \sum_{r=0}^{d} \frac{1}{2d - r + 1} \begin{pmatrix} d\\r \end{pmatrix}
(|x - y|)^r(\mathrm{min}(x, y)  - a)^{2d - r + 1}
$$

$$
\mathbf{K}(\mathbf{x}, \mathbf{y}) = \prod_{j=1}^n K_j(x_j, y_j)
$$

$$
\widetilde{\mathbf{K}}(\mathbf{x}, \mathbf{y}) = \frac{\mathbf{K}(\mathbf{x}, \mathbf{y})}{\sqrt{\mathbf{K}(\mathbf{x}, \mathbf{x})
\mathbf{K}(\mathbf{y}, \mathbf{y})}}
$$

Где под INK-spline ядром понимается $\widetilde{\mathbf{K}}$

## Old version

In [4]:
from functools import partial
import numpy as np
from scipy.misc import comb

glob = 0

def _is_integer(val):
    return isinstance(val, (int, np.int32, np.int64))

def K(X: np.ndarray, Y: np.ndarray, degree: int, a: int) -> np.ndarray:
    gram = np.zeros((X.shape[0], Y.shape[0]))
    verbose_len = X.shape[0]
    for idx, x in enumerate(X):
        S = np.zeros(Y.shape)
        for r in range(degree + 1):
            S += ((x - a)*(Y - a))**r +\
            1.0/(2 * degree - r + 1) * comb(degree, r)*((np.abs(x - Y))**r)*(np.minimum(x, Y) - a)**(2*degree - r + 1)
        gram_row = np.multiply.reduce(S, axis=1)
        gram[idx] = gram_row
    return gram

def K_norm(X: np.ndarray, Y:np.ndarray, degree: int, a: int, axis: str) -> np.ndarray:
    assert axis == "x" or axis =="y", "Invalid axis argument"
    gram = np.zeros((X.shape[0], Y.shape[0]))
    if axis=="x":
        for idx, x in enumerate(X):
            S = np.zeros(x.shape)
            for r in range(degree + 1):
                S += ((x - a)*(x - a))**r +\
                1.0/(2 * degree - r + 1) * comb(degree, r)*(0**r)*(x - a)**(2*degree - r + 1)
            S = np.multiply.reduce(S)
            gram[idx] = S
    elif axis=="y":
        for idx, y in enumerate(Y):
            S = np.zeros(y.shape)
            for r in range(degree + 1):
                S += ((y - a) * (y - a)) ** r + \
                     1.0 / (2 * degree - r + 1) * comb(degree, r) * (0 ** r) * (y - a) ** (2 * degree - r + 1)
            S = np.multiply.reduce(S)
            gram[:, idx:idx+1] = S
    else:
        raise ValueError("Invalid axis name")
    return gram


def ink(x: np.ndarray, y: np.ndarray, degree: int, a: int = -3) -> np.ndarray:
    assert _is_integer(degree) and degree > 0, "Degree must be positive integer"
    assert isinstance(x, np.ndarray) and isinstance(y, np.ndarray), "X and Y must be numpy arrays"
    if len(x.shape) == 1:
        x = x.reshape(1, x.shape[0])
    if len(y.shape) == 1:
        y = y.reshape(1, y.shape[0])
    x[x < a] = a
    y[y < a] = a
    return K(x, y, degree, a) / np.sqrt(K_norm(x, y, degree, a, "x") * K_norm(x, y, degree, a, "y"))

def get_sklearn_ink(degree: int, a: int):
    return partial(ink, degree=degree, a=a)

## New version

In [101]:
def make_fat_xy(X, Y):
    fatX = np.tile(X, (Y.shape[0], 1))
    fatY = np.tile(Y, (1, X.shape[0])).reshape(X.shape[0] * Y.shape[0], X.shape[1])
    return fatX, fatY

def new_K(X: np.ndarray, Y: np.ndarray, degree: int, a: int) -> np.ndarray:
    %time
    fatX, fatY = make_fat_xy(X, Y)
    %time
    first = ((fatX - a)*(fatY - a))
    %time
    absXYdiff = (np.abs(fatX - fatY))
    %time
    XYmin_minus_A = np.minimum(fatX, fatY) - a
    %time
    S = np.zeros(fatX.shape)
    %time
    for r in range(degree + 1):
        S += first**r +\
            1.0/(2 * degree - r + 1) * comb(degree, r)*(absXYdiff**r)*(XYmin_minus_A)**(2*degree - r + 1)
        %time
    S = np.multiply.reduce(S, axis=1)
    %time
    gram = np.transpose(S.reshape((Y.shape[0], X.shape[0])))
    %time
    return gram

def new_K_norm(X: np.ndarray, Y: np.ndarray, degree: int, a: int) -> np.ndarray:
    Sx = np.zeros(X.shape)
    Sy = np.zeros(Y.shape)
    firstX = ((X - a)*(X - a))
    firstY = ((Y - a)*(Y - a))
    for r in range(degree + 1):
        Sx += firstX**r + 1.0/(2 * degree - r + 1) * comb(degree, r)*(0**r)*(X - a)**(2*degree - r + 1)
        Sy += firstY**r + 1.0/(2 * degree - r + 1) * comb(degree, r)*(0**r)*(Y - a)**(2*degree - r + 1)
    Sx = np.multiply.reduce(Sx, axis=1)
    Sy = np.multiply.reduce(Sy, axis=1)
    Sx_matrixed = np.tile(Sx.reshape(Sx.shape[0], 1), Sy.shape[0])
    Sy_matrixed = np.tile(Sy, (Sx.shape[0], 1))
    return np.sqrt(Sx_matrixed * Sy_matrixed)
    
    

def new_ink(X: np.ndarray, Y: np.ndarray, degree: int, a: int = -3) -> np.ndarray:
    assert _is_integer(degree) and degree > 0, "Degree must be positive integer"
    assert isinstance(X, np.ndarray) and isinstance(Y, np.ndarray), "X and Y must be numpy arrays"
    if len(X.shape) == 1:
        X = X.reshape(1, X.shape[0])
    if len(Y.shape) == 1:
        Y = Y.reshape(1, Y.shape[0])
    X[X < a] = a
    Y[Y < a] = a
    return new_K(X, Y, degree, a) / new_K_norm(X, Y, degree, a)

In [48]:
print(ink(np.array([1, 2, 3]), np.array([4, 5, 6]), 1))
print(new_ink(np.array([1, 2, 3]), np.array([4, 5, 6]), 1))

[[ 0.85704509]]


[[ 0.85704509]]


In [103]:
X = np.random.random((1000, 50))
Y = np.random.random((1600, 50))

ink(X, Y, 1)
new_ink(X, Y, 1)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs
CPU times: user 6 µs, sys: 4 µs, total: 10 µs
Wall time: 35 µs
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 20 µs
CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.05 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 7.87 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 16.9 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


array([[ 0.83645066,  0.86809448,  0.86790361, ...,  0.83833956,
         0.83927464,  0.82112949],
       [ 0.84080737,  0.8504549 ,  0.85323693, ...,  0.81908079,
         0.85871453,  0.86491871],
       [ 0.84707083,  0.86639706,  0.82215073, ...,  0.80546032,
         0.84380091,  0.82886667],
       ..., 
       [ 0.85875133,  0.84372679,  0.87082871, ...,  0.80506342,
         0.82443225,  0.86841358],
       [ 0.85483051,  0.87708893,  0.85501822, ...,  0.83499496,
         0.87020118,  0.90230197],
       [ 0.88030099,  0.82386021,  0.84549206, ...,  0.85247555,
         0.84265922,  0.8704307 ]])